## `Библиотеки`

In [1]:
import os
import re
import json
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from db.models import (StreetType, CityType, Address, BelpostAddress,
                    get_database_engine, get_database_url)
from parser import setup_driver, search_postal_code
from rapidfuzz import fuzz

## 0. Загрузка данных

In [2]:
path = os.path.join("private_data", "ko_adresses.xlsx")
data = pd.read_excel(path)
data.head()

,to.street,to.building,soato.imns,soato.oblast,soato.district,soato.sovet,soato.tip,soato.name
0,УЛ. КОЛЬЦЕВАЯ,3,621,Минская,NaN,NaN,г.,Солигорск
1,УЛ. ВЛАДИМИРА ЛЕНИНА,238-3А,313,Витебская,NaN,NaN,г.,Орша
2,УЛ. ИНОГОРОДНЯЯ 3-Я,35,400,NaN,NaN,NaN,г.,Гомель
3,УЛ. БОРИСОВСКАЯ,2Г,603,Минская,Борисовский,Гливинский,д.,Гора
4,"УЛ. СУДОСТРОИТЕЛЬНАЯ, 10, пом. 316",NaN,419,Гомельская,Речицкий,Озерщинский,д.,Озерщина


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264997 entries, 0 to 264996
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   to.street       260015 non-null  object
 1   to.building     228084 non-null  object
 2   soato.imns      264997 non-null  int64 
 3   soato.oblast    134385 non-null  object
 4   soato.district  43071 non-null   object
 5   soato.sovet     39320 non-null   object
 6   soato.tip       264997 non-null  object
 7   soato.name      264997 non-null  object
dtypes: int64(1), object(7)
memory usage: 16.2+ MB


## 1. Предобработка данных 

### 1.1. Замена аббревиатур населенных пунктов на полные слова

In [4]:
data["soato.tip"].value_counts()

soato.tip
г.     221616
аг.     13737
д.      13087
гп       6520
с/с      6246
р-н      1687
п.       1620
рп        246
кп        212
х.         24
пгт         2
Name: count, dtype: int64

In [5]:
replace_dict = {
    "г.": "город",
    "аг.": "агрогородок",
    "гп": "городской поселок",
    "д. ": "деревня",
    "с/с": "сельский совет",
    "р-н": "район",
    "п.": "поселок",
    "рп": "рабочий поселок",
    "кп": "курортный поселок",
    "х.": "хутор",
    "пгт": "поселок городского типа",
}

### 1.2. Универсальный конструктор адреса для поиска

In [6]:
def build_address(
    region: str = None,
    district: str = None,
    sovet: str = None,
    city_type: str = None,
    city_name: str = None,
    street: str = None,
    building: str = None,
) -> str:
    """
    Универсальный конструктор адреса.
    Пропускает отсутствующие поля, не добавляя лишних слов.
    """

    parts = []

    if region:
        parts.append(f"{region} область")

    if district:
        parts.append(f"{district} район")

    if sovet:
        parts.append(f"{sovet} сельсовет")

    if city_name:  # тип города добавляем только если есть название
        if city_type:
            parts.append(f"{city_type} {city_name}")
        else:
            parts.append(city_name)

    if street:
        parts.append(street)

    if building:
        parts.append(building)

    return ", ".join(parts)

### 1.3 Замена аббревиатур в названиях улиц (поле `to.street`)

In [7]:
# Извлекаем возможные сокращения
pattern = r"^([\w\-/.]+)\s"
data["abbr"] = data["to.street"].str.extract(pattern)
abbrs = (data["abbr"].dropna().unique())
abbrs
abbrs_with_dot_dash = [a for a in abbrs if re.search(r'[.\-]', a)]
abbrs_no_digits = [abbr for abbr in abbrs_with_dot_dash if not re.search(r"\d", abbr)]
data["abbr"].value_counts(sort=True)[151:161], abbrs_no_digits[:5]

(abbr
 южнее               9
 Базарная            9
 Центр               8
 маёнтак             8
 ул.50               8
 административное    8
 Смиловичский        8
 ГСПК                8
 2                   8
 ВОЕННЫЙ             8
 Name: count, dtype: int64,
 ['УЛ.', 'ПР.', 'ПЕР.', 'ул.', 'ПЛ.'])

In [8]:
# Загружаем JSON-справочник
def grouped_to_dict(grouped_abbrs: dict[str:list[str]]) -> dict[str:str]:
    abbrs_dict = dict()
    for fullname, abbrs in grouped_abbrs.items():
        for abbr in abbrs:
            abbrs_dict[abbr] = fullname
    return abbrs_dict

abbr_dict = dict()
with open('db/grouped_abbrs.json', 'r', encoding='utf-8') as f:
    grouped_dict = json.load(f)
    abbr_dict = grouped_to_dict(grouped_dict)
list(abbr_dict.items())[:10]

[('улица', 'улица'),
 ('УЛ.', 'улица'),
 ('ул.', 'улица'),
 ('Ул.', 'улица'),
 ('проспект', 'проспект'),
 ('ПР.', 'проспект'),
 ('пр-т', 'проспект'),
 ('пр.', 'проспект'),
 ('просп.', 'проспект'),
 ('переулок', 'переулок')]

In [9]:
def split_street_and_type(df, column, abbr_dict):
    """
    В столбце `column` оставляет только название улицы без аббревиатуры,
    а в новом столбце `streetType` сохраняет расшифровку аббревиатуры (тип улицы).
    abbr_dict: dict, где ключ — аббревиатура, значение — расшифровка.
    """
    # Сортируем аббревиатуры по убыванию длины, чтобы длинные сначала заменялись
    abbrs = sorted(abbr_dict.keys(), key=len, reverse=True)
    # Составляем паттерн для поиска всех аббревиатур (с учётом возможного отсутствия пробела)
    # Аббревиатура должна быть в начале строки, допускается пробел или отсутствие пробела после неё
    abbr_pattern = r'^(' + '|'.join([re.escape(a) for a in abbrs]) + r')\s*'
    abbr_re = re.compile(abbr_pattern, flags=re.IGNORECASE)

    def extract_type_and_street(text):
        if not isinstance(text, str):
            return (None, text)
        m = abbr_re.match(text)
        if m:
            abbr = m.group(1)
            street_type = abbr_dict.get(abbr, abbr_dict.get(abbr.upper(), None))
            # Удаляем аббревиатуру и пробелы после неё
            street_name = text[m.end():].strip()
            return (street_type, street_name)
        else:
            return (None, text.strip())

    res = df.copy()
    res[["streetType", column]] = res[column].apply(lambda x: pd.Series(extract_type_and_street(x)))
    return res

In [10]:
data = split_street_and_type(data, "to.street", abbr_dict)
data.fillna("", inplace=True)
data.sample(10)

,to.street,to.building,soato.imns,soato.oblast,soato.district,soato.sovet,soato.tip,soato.name,abbr,streetType
17579,"ШПИЛЕВСКОГО ПАВЛА, 54, пом. 3Н",,100,,,,г.,Минск,УЛ.,улица
237887,БЕЛОРУССКАЯ Маршрут Витебск-Орша,6а,300,,,,г.,Витебск,УЛ.,улица
224037,БОРИСОВЦА,9,618,Минская,,,г.,Слуцк,УЛ.,улица
147195,ИНДУРСКОЕ,4/2-38,500,,,,г.,Гродно,ШОССЕ,шоссе
49670,МЕДИЦИНСКАЯ,д.13,200,,,,г.,Брест,УЛ.,улица
46818,"МИКРОРАЙОН КОМСОМОЛЬСКИЙ, д.19, пом.9",,761,Могилевская,,,г.,Кричев,УЛ.,улица
115867,ИНДУРСКОЕ,30,500,,,,г.,Гродно,ШОССЕ,шоссе
195553,"Западный промузел,ТЭЦ-4,каб.229",,613,Минская,Минский,Щомыслицкий,с/с,Щомыслицкий,Западный,
37694,ИНТЕРНАЦИОНАЛЬНАЯ,"36,пом.2",100,,,,г.,Минск,УЛ.,улица
151368,ДРОЗДА,25а,100,,,,г.,Минск,УЛ.,улица


In [11]:
# Проверка на наличие аббревиатур
pattern = r"^([\w\-/.]+)\s"
data["abbr"] = data["to.street"].str.extract(pattern)
data["abbr"].value_counts()[:10]

abbr
ВЕРЫ          1748
МАКСИМА       1658
50            1596
ПЕТРА         1011
ВЛАДИМИРА     1008
КУПАЛЫ         962
МАРКСА         952
КАРЛА          944
ВЕЛИКИЙ        922
ТИМИРЯЗЕВА     913
Name: count, dtype: int64

## 2. Поиск адреса в базе (запрос к `belpost.by`)

In [12]:
# Загрузка драйвера и сессии
engine = get_database_engine(echo=False)
session = Session(engine)
driver = setup_driver()

In [13]:
# Составление адреса запроса и запрос к belpost.by
region = district = sovet = city_type = city_name = street = building = address_str = None
addr = None
with session.begin():
    idx = 37967
    addr = session.query(Address).where(Address.id == idx).first()

    streetName = addr.streetName
    streetType = addr.streetType if addr.streetType is not None else ""
    street = streetType + " " + streetName
    building = addr.building
    region = addr.soato_oblast
    district = addr.soato_district
    sovet = addr.soato_sovet
    city_name = addr.soato_name
    city_type = replace_dict.get(addr.soato_tip, "")
    
    # data_row = data.iloc[idx]
    # region = data_row["soato.oblast"]
    # district = data_row["soato.district"]
    # sovet = data_row["soato.sovet"]
    # city_type = replace_dict.get(data_row["soato.tip"], "")
    # city_name = data_row["soato.name"]
    # street = data_row["to.street"]
    # building = data_row["to.building"]

    address_str = build_address(
        region=region,
        district=district,
        sovet=sovet,
        city_type=city_type,
        city_name=city_name,
        street=street,
        building=building,
    )

    print("Составленный адрес: ", address_str)

    res = search_postal_code(driver, address_str)
    res = pd.DataFrame(res, columns=["Почтовый код","Область","Район",
                                     "Город","Улица","Номер дома"])
    session.close()
res

Составленный адрес:  город Гомель, улица РОГАЧЕВСКАЯ, 4
Открываем URL: https://www.belpost.by/Uznatpochtovyykod28indek?search=%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%20%D0%93%D0%BE%D0%BC%D0%B5%D0%BB%D1%8C%2C%20%D1%83%D0%BB%D0%B8%D1%86%D0%B0%20%D0%A0%D0%9E%D0%93%D0%90%D0%A7%D0%95%D0%92%D0%A1%D0%9A%D0%90%D0%AF%2C%204
Ожидание результатов поиска...


,Почтовый код,Область,Район,Город,Улица,Номер дома
0,246003,Гомельская,Гомельский,город Гомель,улица Рогачевская,"2, 2А, 2Б, 2В, 2Г, 4, 6, 1А, 1Б, 5, 5А, 9, 10,..."
1,246025,Гомельская,Гомельский,город Гомель,улица 4 Иногородняя,"3-38, 5/1, 5/2, 10/1, 10/2, 11/1, 11/2, 13А, 2..."
2,246013,Гомельская,Гомельский,город Гомель,улица 4 Техническая,"1-38, 3А, 4/2, 9А, 9Б, 11/1, 11/2, 13А, 15/1, ..."
3,246015,Гомельская,Гомельский,город Гомель,улица 4 Новоселковая,"1-11, 7А, 13"
4,246035,Гомельская,Гомельский,город Гомель,улица 4 Линейная,"(1-9), (2-16)"
5,246049,Гомельская,Гомельский,город Гомель,улица 4 Поперечная,"1-17, 4А, 11А, 12А, 19"
6,246013,Гомельская,Гомельский,город Гомель,переулок 4 Транзитный,"7, (8-70), 22А, 22Б, 64А"
7,246013,Гомельская,Гомельский,город Гомель,переулок 4 Ильича,"1-15, 2А, 2А/2, 2Б, 2В, 7/2, 7/3, 7А, 12А, 13/..."
8,246038,Гомельская,Гомельский,город Гомель,переулок 4 Староволотовской,ВСЕ
9,246035,Гомельская,Гомельский,город Гомель,исправительная колония 4,ВСЕ


## 3. Фильтрация и поиск похожих адресов

### 3.1. Жёсткая фильтрация (по области, району, городу)

In [14]:
def filter_addresses(df: pd.DataFrame, region: str, district: str, city: str) -> pd.DataFrame:
    """Жёсткая фильтрация DataFrame по региону, району, городу и диапазону домов."""
    mask = (
        (df["Область"].astype(str).str.contains(region, case=False, na=False) if region else True) &
        (df["Район"].astype(str).str.contains(district, case=False, na=False) if district else True) &
        (df["Город"].astype(str).str.contains(city, case=False, na=False) if city else True)
    )
    return df[mask]

In [15]:
first_filtered = filter_addresses(res,
                region=region,
                district=district,
                city=city_name)
print("Поисковый адрес: ", address_str)
first_filtered

Поисковый адрес:  город Гомель, улица РОГАЧЕВСКАЯ, 4


,Почтовый код,Область,Район,Город,Улица,Номер дома
0,246003,Гомельская,Гомельский,город Гомель,улица Рогачевская,"2, 2А, 2Б, 2В, 2Г, 4, 6, 1А, 1Б, 5, 5А, 9, 10,..."
1,246025,Гомельская,Гомельский,город Гомель,улица 4 Иногородняя,"3-38, 5/1, 5/2, 10/1, 10/2, 11/1, 11/2, 13А, 2..."
2,246013,Гомельская,Гомельский,город Гомель,улица 4 Техническая,"1-38, 3А, 4/2, 9А, 9Б, 11/1, 11/2, 13А, 15/1, ..."
3,246015,Гомельская,Гомельский,город Гомель,улица 4 Новоселковая,"1-11, 7А, 13"
4,246035,Гомельская,Гомельский,город Гомель,улица 4 Линейная,"(1-9), (2-16)"
5,246049,Гомельская,Гомельский,город Гомель,улица 4 Поперечная,"1-17, 4А, 11А, 12А, 19"
6,246013,Гомельская,Гомельский,город Гомель,переулок 4 Транзитный,"7, (8-70), 22А, 22Б, 64А"
7,246013,Гомельская,Гомельский,город Гомель,переулок 4 Ильича,"1-15, 2А, 2А/2, 2Б, 2В, 7/2, 7/3, 7А, 12А, 13/..."
8,246038,Гомельская,Гомельский,город Гомель,переулок 4 Староволотовской,ВСЕ
9,246035,Гомельская,Гомельский,город Гомель,исправительная колония 4,ВСЕ


### 3.2 Поиск по похожести имени улицы (`rapidfuzz`)

In [16]:
def add_similarity_scores(df: pd.DataFrame, target_string: str, column_name: str, scorer=fuzz.ratio):
    """
    Быстрое вычисление схожести с использованием rapidfuzz
    """
    df = df.copy()
    
    # Вычисляем схожесть для всех строк сразу
    scores = [scorer(str(x).lower(), str(target_string).lower()) for x in df[column_name]]
    
    df['similarity_score'] = scores
    df.sort_values(by="similarity_score", ascending=False, inplace=True)
    return df

In [17]:
print("Улица: ", street)
data_scores = add_similarity_scores(first_filtered,
                      target_string=street,
                      column_name="Улица")
data_scores

Улица:  улица РОГАЧЕВСКАЯ


,Почтовый код,Область,Район,Город,Улица,Номер дома,similarity_score
0,246003,Гомельская,Гомельский,город Гомель,улица Рогачевская,"2, 2А, 2Б, 2В, 2Г, 4, 6, 1А, 1Б, 5, 5А, 9, 10,...",100.000000
2,246013,Гомельская,Гомельский,город Гомель,улица 4 Техническая,"1-38, 3А, 4/2, 9А, 9Б, 11/1, 11/2, 13А, 15/1, ...",66.666667
3,246015,Гомельская,Гомельский,город Гомель,улица 4 Новоселковая,"1-11, 7А, 13",64.864865
5,246049,Гомельская,Гомельский,город Гомель,улица 4 Поперечная,"1-17, 4А, 11А, 12А, 19",57.142857
4,246035,Гомельская,Гомельский,город Гомель,улица 4 Линейная,"(1-9), (2-16)",54.545455
1,246025,Гомельская,Гомельский,город Гомель,улица 4 Иногородняя,"3-38, 5/1, 5/2, 10/1, 10/2, 11/1, 11/2, 13А, 2...",50.000000
8,246038,Гомельская,Гомельский,город Гомель,переулок 4 Староволотовской,ВСЕ,36.363636
7,246013,Гомельская,Гомельский,город Гомель,переулок 4 Ильича,"1-15, 2А, 2А/2, 2Б, 2В, 7/2, 7/3, 7А, 12А, 13/...",29.411765
9,246035,Гомельская,Гомельский,город Гомель,исправительная колония 4,ВСЕ,29.268293
6,246013,Гомельская,Гомельский,город Гомель,переулок 4 Транзитный,"7, (8-70), 22А, 22Б, 64А",26.315789


### 3.3. Поиск по номеру дома (содержится ли в списке домов)

In [18]:
def normalize_house(house: str):
    """Разбивает дом на число, букву/корпус (если есть)."""
    house = house.strip().upper()
    # Пример: 71/1 → основа 71, корпус 1
    m = re.match(r"(\d+)([А-ЯA-Z]*)?(?:/(\d+))?", house)
    if not m:
        return None, None, None
    num = int(m.group(1))
    letter = m.group(2) or None
    korp = m.group(3) or None
    return num, letter, korp


def house_in_range(house: str, rule: str) -> bool:
    """Проверка: принадлежит ли дом правилу из списка домов."""
    if not house or not rule:
        return False
    
    house_num, house_letter, house_korp = normalize_house(house)
    if house_num is None:
        return False
    
    rule = rule.strip().upper()
    
    if rule == "ВСЕ":
        return True

    # Разделяем правила через запятую
    parts = [p.strip().upper() for p in rule.split(",")]
    for part in parts:
        # диапазон чёт/нечет
        m = re.match(r"\((\d+)-(\d+)\)", part)
        if m:
            start, end = int(m.group(1)), int(m.group(2))
            if house_num % 2 == start % 2 and start <= house_num <= end:
                return True
            continue

        # обычный диапазон
        m = re.match(r"^(\d+)-(\d+)$", part)
        if m:
            start, end = int(m.group(1)), int(m.group(2))
            if start <= house_num <= end:
                return True
            continue

        # конкретный номер (включая буквы и корпуса)
        if part == house.upper():
            return True

    return False

In [19]:
target_house = building
print("Номер дома: ", target_house)
data_scores["house_match"] = data_scores["Номер дома"].apply(lambda r: house_in_range(target_house, r))
data_scores

Номер дома:  4


,Почтовый код,Область,Район,Город,Улица,Номер дома,similarity_score,house_match
0,246003,Гомельская,Гомельский,город Гомель,улица Рогачевская,"2, 2А, 2Б, 2В, 2Г, 4, 6, 1А, 1Б, 5, 5А, 9, 10,...",100.000000,True
2,246013,Гомельская,Гомельский,город Гомель,улица 4 Техническая,"1-38, 3А, 4/2, 9А, 9Б, 11/1, 11/2, 13А, 15/1, ...",66.666667,True
3,246015,Гомельская,Гомельский,город Гомель,улица 4 Новоселковая,"1-11, 7А, 13",64.864865,True
5,246049,Гомельская,Гомельский,город Гомель,улица 4 Поперечная,"1-17, 4А, 11А, 12А, 19",57.142857,True
4,246035,Гомельская,Гомельский,город Гомель,улица 4 Линейная,"(1-9), (2-16)",54.545455,True
1,246025,Гомельская,Гомельский,город Гомель,улица 4 Иногородняя,"3-38, 5/1, 5/2, 10/1, 10/2, 11/1, 11/2, 13А, 2...",50.000000,True
8,246038,Гомельская,Гомельский,город Гомель,переулок 4 Староволотовской,ВСЕ,36.363636,True
7,246013,Гомельская,Гомельский,город Гомель,переулок 4 Ильича,"1-15, 2А, 2А/2, 2Б, 2В, 7/2, 7/3, 7А, 12А, 13/...",29.411765,True
9,246035,Гомельская,Гомельский,город Гомель,исправительная колония 4,ВСЕ,29.268293,True
6,246013,Гомельская,Гомельский,город Гомель,переулок 4 Транзитный,"7, (8-70), 22А, 22Б, 64А",26.315789,False


### 3.4. Выбор лучшего результата

In [20]:
cond = data_scores["house_match"].sum() != 0
res = data_scores[data_scores["house_match"]] if cond else data_scores # выбираем только те строки, где дом совпал
max_score = res['similarity_score'].max()
res[res['similarity_score'] == max_score] # выбираем строку с максимальным значением к-та совпадения

,Почтовый код,Область,Район,Город,Улица,Номер дома,similarity_score,house_match
0,246003,Гомельская,Гомельский,город Гомель,улица Рогачевская,"2, 2А, 2Б, 2В, 2Г, 4, 6, 1А, 1Б, 5, 5А, 9, 10,...",100.0,True
